In [2]:
#--- crawling ---#
import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import urllib.parse as par
from bs4 import BeautifulSoup as BS
import requests
from selenium import webdriver

#### _테스트로 1페이지만 가져오기_

In [251]:
pn = str(1)
url = "https://www.gfauction.info/auction/list.html?page="+pn+"&listnum=100&orderby=&special=&court1=&damdang1=&syear=&sno=&gamMin=0&gamMax=0&eng=4%2C7&uchal_min=&uchal_max=&sday_s=&sday_e=&lowMin=0&lowMax=0&sido1=11&gugun1=&dong1=&bunji=&sagunname=&barea_min=&barea_max=&larea_min=&larea_max=&yongdo=01&special=&addr="
#print(url)
page = req.urlopen(url)
print(page)

In [252]:
soup = BS(page, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html lang="ko">
 <head>
  <meta charset="utf-8"/>
  <title>
   굿프렌드 경매 | 경매 플랫폼
  </title>
  <!--<meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=5.0,minimum-scale=0.1,user-scalable=yes,target-densitydpi=medium-dpi" />-->
  <meta content="굿프렌드, 굿프렌드경매, 굿프렌드경매학원, 법원경매, 부동산공매, 경매플랫폼, AI권리분석, 실시간낙찰결과, 경매데이터분석, 경매강좌" name="description"/>
  <meta content="굿프렌드,굿프렌드경매,굿프렌드경매학원,법원경매,부동산경매,권리분석,경매정보,경매물건,경매일정,경매결과,아파트경매,경락잔금대출,경매사이트,법원경매자료,입찰정보,입찰결과,낙찰가,부동산정보,재개발,재건축,뉴타운,법원경매정보,경매자료,경매양식,경매서식,세대열람,재테크,경매강좌동영상경매강좌,경매교육,경매가맹점,경매컨설팅,경매대행,부동산재테크,전자지도,지적도" name="keywords"/>
  <meta content="4960bab9a95aa17ecaece910626a719c19911f53" name="naver-site-verification">
   <meta content="width=1300, user-scalable=yes" name="viewport">
    <meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
    <meta content="website" property="og:type"/>
    <meta content="굿프렌드경매 부동산경매정보" property="og:title"/>
    <meta content="법원경매, 부동산공매, 경매플랫폼, AI권리분석, 실시간

In [253]:
body = soup.find_all("td", style="border-bottom:1px solid #D9D7D7")

In [ ]:
"""
변수명: 

담당법원(office), 사건번호(num), 건물형태(build), 주소(addr), 
건물면적(build_area), 토지면적(land_area), 기타사항(comment), 
감정가(price), 최저가(lowest), 낙찰가(result), 
매각기일(date), 상태(status), 최저비율(low_per), 낙찰비율(res_per)
"""

In [246]:
data = np.reshape(body, (-1, 6))
df = pd.DataFrame(columns=["office", "num", "cat", "addr", "build_area", "land_area", "price", "lowest", "result", "date", "status", "low_per", "res_per", "comment"])

for idx in range(0,len(data)):
    office = data[idx,2].find_all("p")[0].get_text().strip(" ").split(" ")[0]
    num = data[idx,2].find_all("p")[0].get_text().strip(" ").split(" ")[1]
    cat = data[idx,2].find_all("p")[1].find("strong").get_text().strip(" []")
    addr = data[idx,2].find_all("p")[1].get_text()[6:]
    build_area = data[idx,2].find_all("p")[2].get_text().replace("\t","").split("|")[0].strip(" ")[3:]
    land_area = data[idx,2].find_all("p")[2].get_text().replace("\t","").split("|")[1].strip(" ")[3:]
    if data[idx,2].find("p", class_="f_red") == None:
        comment = ""
    else:
        comment = data[idx,2].find("p", class_="f_red").get_text().strip(" []")
    price = data[idx,3].find("p","ico_emo").get_text()
    lowest = data[idx,3].find("p", "ico_low").get_text()
    if data[idx,5].get_text().replace("\r\n","").replace("\t","")[:2] == "매각":
        result = data[idx,3].find("p","ico_maegak f_red").get_text()
    else:
        result = ""
    date = data[idx,4].get_text().strip(" ")
    status = data[idx,5].get_text().replace("\r\n","").replace("\t","")[:2]
    low_per = data[idx,5].get_text().replace("\r\n","").replace("\t","")[2:7]
    res_per = data[idx,5].get_text().replace("\r\n","").replace("\t","")[9:14]
    
    test2 = [office, num, cat, addr, build_area, land_area, comment, price, lowest, result, date, status, low_per, res_per]
    df = df.append(pd.DataFrame([test2], columns=["office", "num", "cat", "addr", "build_area", "land_area", "comment","price", "lowest", "result", "date", "status", "low_per", "res_per"]), ignore_index=True)
    
df

C:\Users\wlsth\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,office,num,cat,addr,build_area,land_area,price,lowest,result,date,status,low_per,res_per,comment
0,서울남부3계,2020-5706,아파트,"서울특별시 양천구 신월동 98-9, 제2층 제202호 (신월동,하나하이츠)",53.58㎡ (16.21평),30.72㎡ (9.29평),"212,000,000","135,680,000","188,100,000",2021-07-21(10:00),매각,(64%),89%)\n,
1,서울남부4계,2020-110424,아파트,"서울특별시 영등포구 영등포동1가 75-4, 제9층 제902호 (영등포동1가, 메트로가든)",12.97㎡ (3.92평),5.22㎡ (1.58평),"143,000,000","114,400,000","120,660,000",2021-07-14(10:00),매각,(80%),84%)\n,선순위전세권
2,서울서부1계,2020-3808,아파트,"서울특별시 마포구 서교동 490, 아파트동 14층101동1406호 (서교동,메세나폴리스)",142.034㎡ (42.97평),26.39㎡ (7.98평),"1,733,000,000","1,733,000,000","1,869,999,999",2021-07-13(10:00),매각,(100%,(108%,법정지상권
3,서울중앙2계,2020-1845,아파트,"서울특별시 강남구 청담동 69-18, 101동 12층1203호 (청담동,청담동양파라곤)",171.1005㎡ (51.76평),87.666㎡ (26.52평),"3,440,000,000","3,440,000,000","3,953,999,000",2021-07-08(10:00),매각,(100%,(115%,


#### (1) 매각완료된 데이터

In [248]:
from tqdm import tqdm_notebook

In [277]:
pn = range(1,27)
df = pd.DataFrame(columns=["office", "num", "cat", "addr", "build_area", "land_area", "price", "lowest", "result", "date", "status", "low_per", "res_per", "comment"])
for i in tqdm_notebook(pn):
    url = "https://www.gfauction.info/auction/list.html?page="+str(i)+"&listnum=100&orderby=&special=&court1=&damdang1=&syear=&sno=&gamMin=0&gamMax=0&eng=4%2C7&uchal_min=&uchal_max=&sday_s=&sday_e=&lowMin=0&lowMax=0&sido1=11&gugun1=&dong1=&bunji=&sagunname=&barea_min=&barea_max=&larea_min=&larea_max=&yongdo=01&special=&addr="
    page = req.urlopen(url)
    soup = BS(page, "html.parser")
    body = soup.find_all("td", style="border-bottom:1px solid #D9D7D7")
    data = np.reshape(body, (-1, 6))
    
    for idx in range(0,len(data)):
        office = data[idx,2].find_all("p")[0].get_text().replace("\t","").strip(" ").split(" ")[0]
        num = data[idx,2].find_all("p")[0].get_text().replace("\t","").strip(" ").split(" ")[1]
        cat = data[idx,2].find_all("p")[1].find("strong").get_text().replace("\t","").strip(" []")
        addr = data[idx,2].find_all("p")[1].get_text().replace("\t","")[6:]
        build_area = data[idx,2].find_all("p")[2].get_text().replace("\t","").split("|")[0].strip(" ")[3:]
        if data[idx,2].find_all("p")[2].get_text().find("|") == -1:
            land_area = ""
        else:
            land_area = data[idx,2].find_all("p")[2].get_text().replace("\t","").split("|")[1].strip(" ")[3:]
        if data[idx,2].find("p", class_="f_red") == None:
            comment = ""
        else:
            comment = data[idx,2].find("p", class_="f_red").get_text().strip(" []")
        price = data[idx,3].find("p","ico_emo").get_text()
        lowest = data[idx,3].find("p", "ico_low").get_text()
        if data[idx,5].get_text().replace("\r\n","").replace("\t","")[:2] == "매각":
            result = data[idx,3].find("p","ico_maegak f_red").get_text()
        else:
            result = ""
        date = data[idx,4].get_text().strip(" ")
        status = data[idx,5].get_text().replace("\r\n","").replace("\t","").split("(")[0].strip(" )\n")
        low_per = data[idx,5].get_text().replace("\r\n","").replace("\t","").split("(")[1].strip(" )\n")
        if data[idx,5].get_text().replace("\r\n","").replace("\t","")[:2] == "매각":
            res_per = data[idx,5].get_text().replace("\r\n","").replace("\t","").split("(")[2].strip(" )\n")
        else:
            res_per = ""

        item = [office, num, cat, addr, build_area, land_area, comment, price, lowest, result, date, status, low_per, res_per]
        df = df.append(pd.DataFrame([item], columns=["office", "num", "cat", "addr", "build_area", "land_area", "comment","price", "lowest", "result", "date", "status", "low_per", "res_per"]), ignore_index=True)

df

Wall time: 0 ns


C:\Users\wlsth\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,office,num,cat,addr,build_area,land_area,price,lowest,result,date,status,low_per,res_per,comment
0,서울남부3계,2020-5706,아파트,"서울특별시 양천구 신월동 98-9, 제2층 제202호 (신월동,하나하이츠)",53.58㎡ (16.21평),30.72㎡ (9.29평),"212,000,000","135,680,000","188,100,000",2021-07-21(10:00),매각,64%,89%,
1,서울남부4계,2020-110424,아파트,"서울특별시 영등포구 영등포동1가 75-4, 제9층 제902호 (영등포동1가, 메트로가든)",12.97㎡ (3.92평),5.22㎡ (1.58평),"143,000,000","114,400,000","120,660,000",2021-07-14(10:00),매각,80%,84%,선순위전세권
2,서울서부1계,2020-3808,아파트,"서울특별시 마포구 서교동 490, 아파트동 14층101동1406호 (서교동,메세나폴리스)",142.034㎡ (42.97평),26.39㎡ (7.98평),"1,733,000,000","1,733,000,000","1,869,999,999",2021-07-13(10:00),매각,100%,108%,법정지상권
3,서울중앙2계,2020-1845,아파트,"서울특별시 강남구 청담동 69-18, 101동 12층1203호 (청담동,청담동양파라곤)",171.1005㎡ (51.76평),87.666㎡ (26.52평),"3,440,000,000","3,440,000,000","3,953,999,000",2021-07-08(10:00),매각,100%,115%,
4,서울중앙2계,2020-101356,아파트,"서울특별시 서초구 반포동 88-4, 2층 202호 (반포동,서래아르드빌)",200.4㎡ (60.62평),86.73㎡ (26.24평),"1,860,000,000","1,190,400,000","1,595,000,000",2021-07-08(10:00),매각,64%,86%,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2585,서울북부3계,2015-100822,아파트,"서울특별시 중랑구 망우동 0,동 3층 307호 \r\n[도로명주소] 서울특별시 ...",85㎡ (25.71평),25㎡ (7.56평),"270,000,000","216,000,000",,2015-10-12(10:00),배당종결,80%,,
2586,서울중앙8계,2014-28061,아파트,서울특별시 강남구 삼성동 130 삼성이안애빌 3층 304호,107㎡ (32.37평),55㎡ (16.64평),"780,000,000","499,200,000",,2015-10-08(10:00),배당종결,64%,,
2587,서울남부9계,2015-6729,아파트,"서울 양천구 신정동 1314, 양천중앙하이츠 102동 9층 906호 \r\n서...",135㎡ (40.84평),67㎡ (20.27평),"700,000,000","560,000,000",,2015-10-06(10:00),배당종결,80%,,
2588,서울북부1계,2015-1437,아파트,"서울특별시 노원구 중계동 0,동 8층 805호 \r\n[도로명주소] 서울특별시 ...",135㎡ (40.84평),66㎡ (19.96평),"760,000,000","608,000,000",,2015-10-05(10:00),배당종결,80%,,


#### _중간 데이터 저장(매각완료된 데이터)_

In [291]:
df.to_csv("data/auction/부동산경매데이터.csv", index=False)

#### (2) 매각진행 중인 데이터

In [279]:
url= "https://www.gfauction.info/auction/list.html?page=1&listnum=100&orderby=&special=&court1=&damdang1=&syear=&sno=&gamMin=0&gamMax=0&eng=12%2C15&uchal_min=&uchal_max=&sday_s=&sday_e=&lowMin=0&lowMax=0&sido1=11&gugun1=&dong1=&bunji=&sagunname=&barea_min=&barea_max=&larea_min=&larea_max=&yongdo=01&special=&addr="
page = req.urlopen(url)
soup = BS(page, "html.parser")
body = soup.find_all("td", style="border-bottom:1px solid #D9D7D7")
data = np.reshape(body, (-1, 6))
df2 = pd.DataFrame(columns=["office", "num", "cat", "addr", "build_area", "land_area", "price", "lowest", "result", "date", "status", "low_per", "res_per", "comment"])

for idx in range(0,len(data)):
    office = data[idx,2].find_all("p")[0].get_text().replace("\t","").strip(" ").split(" ")[0]
    num = data[idx,2].find_all("p")[0].get_text().replace("\t","").strip(" ").split(" ")[1]
    cat = data[idx,2].find_all("p")[1].find("strong").get_text().replace("\t","").strip(" []")
    addr = data[idx,2].find_all("p")[1].get_text().replace("\t","")[6:]
    build_area = data[idx,2].find_all("p")[2].get_text().replace("\t","").split("|")[0].strip(" ")[3:]
    if data[idx,2].find_all("p")[2].get_text().find("|") == -1:
        land_area = ""
    else:
        land_area = data[idx,2].find_all("p")[2].get_text().replace("\t","").split("|")[1].strip(" ")[3:]
    if data[idx,2].find("p", class_="f_red") == None:
        comment = ""
    else:
        comment = data[idx,2].find("p", class_="f_red").get_text().strip(" []")
    price = data[idx,3].find("p","ico_emo").get_text()
    lowest = data[idx,3].find("p", "ico_low").get_text()
    if data[idx,5].get_text().replace("\r\n","").replace("\t","")[:2] == "매각":
        result = data[idx,3].find("p","ico_maegak f_red").get_text()
    else:
        result = ""
    date = data[idx,4].get_text().strip(" ")
    status = data[idx,5].get_text().replace("\r\n","").replace("\t","").split("(")[0].strip(" )\n")
    low_per = data[idx,5].get_text().replace("\r\n","").replace("\t","").split("(")[1].strip(" )\n")
    if data[idx,5].get_text().replace("\r\n","").replace("\t","")[:2] == "매각":
        res_per = data[idx,5].get_text().replace("\r\n","").replace("\t","").split("(")[2].strip(" )\n")
    else:
        res_per = ""

    item = [office, num, cat, addr, build_area, land_area, comment, price, lowest, result, date, status, low_per, res_per]
    df2 = df2.append(pd.DataFrame([item], columns=["office", "num", "cat", "addr", "build_area", "land_area", "comment","price", "lowest", "result", "date", "status", "low_per", "res_per"]), ignore_index=True)

df2

C:\Users\wlsth\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,office,num,cat,addr,build_area,land_area,price,lowest,result,date,status,low_per,res_per,comment
0,서울동부1계,2020-52099,아파트,"서울특별시 송파구 잠실동 22, 265동 22층2201호 (잠실동,리센츠)",124.22㎡ (37.58평),58.63㎡ (17.74평),"2,520,000,000","2,520,000,000",,2021-08-02(10:00)입찰 7일전\t,신건,100%,,
1,서울동부1계,2020-55340,아파트,"서울특별시 성동구 성수동1가 718, 103동 29층2902호 (성수동1가,트리마제)",84.5402㎡ (25.57평),37.6321㎡ (11.38평),"3,100,000,000","2,480,000,000",,2021-08-02(10:00)입찰 7일전\t,유찰1회,80%,,
2,서울동부1계,2020-56640,아파트,"서울특별시 광진구 구의동 587-58, 3층303호",165.14㎡ (49.95평),74.3㎡ (22.48평),"1,330,000,000","1,064,000,000",,2021-08-02(10:00)입찰 7일전\t,유찰1회,80%,,
3,서울중앙10계,2020-3254,아파트,"서울특별시 동작구 상도동 418, 101동 9층906호 (상도동,쌍용스윗닷홈)",59.578㎡ (18.02평),38.427㎡ (11.62평),"736,000,000","736,000,000",,2021-08-04(10:00)입찰 9일전\t,신건,100%,,
4,서울중앙10계,2020-3339,아파트,"서울특별시 강남구 개포동 1280, 213동 13층1304호 (개포동,래미안블레스티지)",29.72㎡ (8.99평),29.72㎡ (8.99평),"1,585,000,000","1,585,000,000",,2021-08-04(10:00)입찰 9일전\t,신건,100%,,"지분매각,토지및건물 지분매각"
5,서울동부4계,2020-55715,아파트,"서울특별시 성동구 홍익동 53, 101동 10층1005호 (홍익동,청계대주파크빌)",,,"391,000,000","391,000,000",,2021-08-09(10:00)입찰 14일전\t,신건,100%,,
6,서울서부6계,2019-52798,아파트,"서울특별시 서대문구 연희동 194-30, 3층비312호 (연희동,브라운스톤 연희)",17.01㎡ (5.15평),7.396㎡ (2.24평),"195,000,000","10,720,000",,2021-08-10(10:00)입찰 15일전\t,유찰,5%,,대항력있는임차인
7,서울서부3계,2019-54121,아파트,"서울특별시 서대문구 연희동 194-30, 3층비314호 (연희동,브라운스톤 연희)",16.62㎡ (5.03평),7.226㎡ (2.19평),"190,000,000","13,058,000",,2021-08-10(10:00)입찰 15일전\t,유찰12회,7%,,대항력있는임차인
8,서울중앙21계,2020-107934,아파트,"서울특별시 관악구 신림동 1563-82, 1동 5층502호",84.79㎡ (25.65평),39.44㎡ (11.93평),"523,000,000","523,000,000",,2021-08-10(10:00)입찰 15일전\t,신건,100%,,
9,서울북부2계,2020-104552,아파트,"서울특별시 도봉구 쌍문동 711, 8층811호 (쌍문동,도봉금강아미움파크타운)",68.2097㎡ (20.63평),21.502㎡ (6.5평),"391,000,000","312,800,000",,2021-08-16(10:00)입찰 21일전\t,유찰1회,80%,,대항력있는임차인


#### _매각완료된 데이터, 매각진행 중인 데이터 취합_

In [293]:
df3 = pd.concat([df2, df],ignore_index=True)

In [294]:
df3

,office,num,cat,addr,build_area,land_area,price,lowest,result,date,status,low_per,res_per,comment
0,서울동부1계,2020-52099,아파트,"서울특별시 송파구 잠실동 22, 265동 22층2201호 (잠실동,리센츠)",124.22㎡ (37.58평),58.63㎡ (17.74평),"2,520,000,000","2,520,000,000",,2021-08-02(10:00)입찰 7일전\t,신건,100%,,
1,서울동부1계,2020-55340,아파트,"서울특별시 성동구 성수동1가 718, 103동 29층2902호 (성수동1가,트리마제)",84.5402㎡ (25.57평),37.6321㎡ (11.38평),"3,100,000,000","2,480,000,000",,2021-08-02(10:00)입찰 7일전\t,유찰1회,80%,,
2,서울동부1계,2020-56640,아파트,"서울특별시 광진구 구의동 587-58, 3층303호",165.14㎡ (49.95평),74.3㎡ (22.48평),"1,330,000,000","1,064,000,000",,2021-08-02(10:00)입찰 7일전\t,유찰1회,80%,,
3,서울중앙10계,2020-3254,아파트,"서울특별시 동작구 상도동 418, 101동 9층906호 (상도동,쌍용스윗닷홈)",59.578㎡ (18.02평),38.427㎡ (11.62평),"736,000,000","736,000,000",,2021-08-04(10:00)입찰 9일전\t,신건,100%,,
4,서울중앙10계,2020-3339,아파트,"서울특별시 강남구 개포동 1280, 213동 13층1304호 (개포동,래미안블레스티지)",29.72㎡ (8.99평),29.72㎡ (8.99평),"1,585,000,000","1,585,000,000",,2021-08-04(10:00)입찰 9일전\t,신건,100%,,"지분매각,토지및건물 지분매각"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,서울북부3계,2015-100822,아파트,"서울특별시 중랑구 망우동 0,동 3층 307호 \r\n[도로명주소] 서울특별시 ...",85㎡ (25.71평),25㎡ (7.56평),"270,000,000","216,000,000",NaN,2015-10-12(10:00),배당종결,80%,NaN,NaN
2609,서울중앙8계,2014-28061,아파트,서울특별시 강남구 삼성동 130 삼성이안애빌 3층 304호,107㎡ (32.37평),55㎡ (16.64평),"780,000,000","499,200,000",NaN,2015-10-08(10:00),배당종결,64%,NaN,NaN
2610,서울남부9계,2015-6729,아파트,"서울 양천구 신정동 1314, 양천중앙하이츠 102동 9층 906호 \r\n서...",135㎡ (40.84평),67㎡ (20.27평),"700,000,000","560,000,000",NaN,2015-10-06(10:00),배당종결,80%,NaN,NaN
2611,서울북부1계,2015-1437,아파트,"서울특별시 노원구 중계동 0,동 8층 805호 \r\n[도로명주소] 서울특별시 ...",135㎡ (40.84평),66㎡ (19.96평),"760,000,000","608,000,000",NaN,2015-10-05(10:00),배당종결,80%,NaN,NaN


In [295]:
df3.to_csv("data/auction/부동산경매데이터_전체.csv", index=False)

In [296]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   office      2613 non-null   object
 1   num         2613 non-null   object
 2   cat         2613 non-null   object
 3   addr        2613 non-null   object
 4   build_area  2589 non-null   object
 5   land_area   2512 non-null   object
 6   price       2613 non-null   object
 7   lowest      2613 non-null   object
 8   result      837 non-null    object
 9   date        2613 non-null   object
 10  status      2613 non-null   object
 11  low_per     2613 non-null   object
 12  res_per     837 non-null    object
 13  comment     279 non-null    object
dtypes: object(14)
memory usage: 285.9+ KB


#### (3) 아파트 실거래가 데이터

In [ ]:
from glob import glob